In [ ]:
# data manipulation and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for saving the pipeline
import joblib

#for loading json file
import json

# from Scikit-learn
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler


import preprocessors as pp

In [ ]:
# setting pandas to display all columns:
pd.set_option('display.max_column', None)

In [ ]:
# load config.json file

with open('config.json') as config_file:
    config = json.load(config_file)

In [ ]:
## Load data from the Google sheet directly:
sheet_id = config['sheet_id']

sheet_name = config['sheet_name']

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
# load dataset
#data = pd.read_csv('data.csv')

data = pd.read_csv(url)

# rows and columns of the data
print(data.shape)

# visualise the dataset
data.head()

Answers to the 'What utilities are available' question:

['electricity', 'Refuse disposal', 'Constant running water in rooms', 'Security post', 'Cleaners', 'Lodge generator(for electricity)', 'Solar']

In [ ]:
# loading the previous dataset 'data.csv'

df = pd.read_csv('data.csv')

In [ ]:
df.head(1)

In [ ]:
df.columns

In [ ]:
data.head(1)

In [ ]:
# Attempting to split the answers in the 'What utilities are availale?' section into columns that can be used for modelling

all_utilities = set()
for row in data['What utilities are available?']:
    if isinstance(row, str):
        utilities = [utility.strip() for utility in row.split(',')]
        all_utilities.update(utilities)


In [ ]:
all_utilities

Lodge generator and Lodge generator(for electricity) should hold the same information but from above would most likely be split into seperate columns, 

lets confirm if there are entries in the data that had these two occurrences:

In [ ]:
set(data['What utilities are available?'].values)

Clearly there are, so this should be handled during data cleaning phase of the model building:

In [ ]:
# Proceeding with engineering columns for the answers to 'What Utilities are available'

# Create columns for each utility and initialize with False
for utility in all_utilities:
    data[utility] = False

# Update the columns based on the 'What utilities are available?' column
for index, row in data.iterrows():
    if pd.notna(row['What utilities are available?']):
        utilities = [utility.strip() for utility in row['What utilities are available?'].split(',')]
        for utility in utilities:
            data.at[index, utility] = True


In [ ]:
# dropping that column since its entries has been made into columns
data.drop('What utilities are available?',axis=1, inplace=True)

In [ ]:
data.head(1)

In [ ]:
data

In [ ]:
data = data.drop('Timestamp', axis=1)

In [ ]:
# the resulting columns:
data.columns

The columns has now been split, The Model Building process can now resume for this engineered dataset.

In [ ]:
data.columns = ['Name', 'Rent', 'StrtName', 'Storeys', 'Cheaperflrs', 'LgCond', 'Age', 'GenHouse', 'Parking', 'Distance',
       'Location', 'RdCond', 'SecurityLvl', 'RmSize', 'RmCond', 'Wdrobe',
       'Finishing', 'Balcony', 'KitchenSize', 'BathrmSize', 'BalcnySize', 
       'RefDisposal', 'LodgeGen', 'SecPost', 'Solar', 'RunWater', 'Cleaners',
       'Electricity', 'ElecLodgeGen']

In [ ]:
# Convert the "rent" column to numeric (ignore errors for non-numeric values)
data['Rent'] = pd.to_numeric(data['Rent'], errors='coerce')

# Apply the condition to the "rent" column using a mask
mask = (data['Rent'].notna()) & (data['Rent'] < 1000)

# Multiply the values that meet the condition by 1000
data.loc[mask, 'Rent'] *= 1000

In [ ]:
# Create a new 'Id' column with row IDs
data['ID'] = range(1, len(data) + 1)

# Reorder the columns so that 'ID' is the first column
data = data[['ID'] + [col for col in data if col != 'ID']]

In [ ]:
data

In [ ]:
# Iterate through each column to handle NaN values
for col in data.columns:
    if data[col].dtype in [int, float]:
        # Replace NaN with the column's average
        data[col].fillna(data[col].mean(), inplace=True)
    elif data[col].dtype == 'object':
        # Replace NaN with the most common string value in the same column
        most_common = data[col].mode()[0]
        data[col].fillna(most_common, inplace=True)
    elif data[col].dtype == 'bool':
        # Replace NaN with the most frequent boolean value
        most_frequent = data[col].mode()[0]
        data[col].fillna(most_frequent, inplace=True)

In [ ]:
data

## Model Development:

- Create a base model using the data retrieved from the URL directly after proper data cleaning and preprocessing

In [ ]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['ID', 'Rent'], axis=1), # predictive variables
    data['Rent'], # target
    test_size=0.3, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

In [ ]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [ ]:
# numerical variables with NA in train set
NUMERICAL_VARS_WITH_NA = ['KitchenSize', 'BathrmSize', 'BalcnySize']

# variables to map
SOME_VARS = ['Cheaperflrs', 'GenHouse', 'Parking', 'Wdrobe', 'Balcony']

MORE_VARS = ['RefDisposal', 'LodgeGen', 'SecPost', 'Solar', 'RunWater', 'Cleaners',
       'Electricity', 'ElecLodgeGen']

# categorical variables to encode
LOCATION_VARS = ['Location']

FINISHING_VARS = ['Finishing']

# variable mappings
SOME_MAPPINGS = {'No':1, 'I don\'t know':2,
                 'Yes':3}

MORE_MAPPINGS = {True:1, False:2}

LOCATION_MAPPINGS = {'Near Eziobodo Gate':1, 'Near Sekani':2,
                     'Around John Paul\'s Kitchen':3, 'Dombolo':4,
                     'Eziobodo Elu':5}

FINISHING_MAPPINGS = {'Tiles':1, 'Cement':2}

# the selected variables
FEATURES = ['Storeys', 'Cheaperflrs', 'LgCond', 'Age', 'GenHouse', 'Parking', 'Distance',
       'Location', 'RdCond', 'SecurityLvl', 'RmSize', 'RmCond', 'Wdrobe',
       'Finishing', 'Balcony', 'KitchenSize', 'BathrmSize', 'BalcnySize', 
       'RefDisposal', 'LodgeGen', 'SecPost', 'Solar', 'RunWater', 'Cleaners',
       'Electricity', 'ElecLodgeGen']





In [ ]:
X_train = X_train[FEATURES]
X_test = X_test[FEATURES]

X_train.shape, X_test.shape

In [ ]:
price_pipe = Pipeline([
        # === mappers ===
    ('mapper_SOME', pp.Mapper(
        variables=SOME_VARS, mappings=SOME_MAPPINGS)),
    
    ('mapper_LOCATION', pp.Mapper(
        variables=LOCATION_VARS, mappings=LOCATION_MAPPINGS)),
    ('mapper_FINISHING', pp.Mapper(
        variables=FINISHING_VARS, mappings=FINISHING_MAPPINGS)),
    ('mapper_MORE', pp.Mapper(
        variables=MORE_VARS, mappings=MORE_MAPPINGS)),
        
    ('scaler', MinMaxScaler()),
#     ('selector', SelectFromModel(Lasso(alpha=0.001, random_state=0))),
    ('Lasso', Lasso(alpha=0.044, random_state=0))
])

In [ ]:
# train the pipeline
price_pipe.fit(X_train, y_train)

In [ ]:
# evaluate the model:
# ====================

# make predictions for train set
pred = price_pipe.predict(X_train)

# determine mse, rmse and r2
print('train mse: {}'.format(int(
    mean_squared_error(np.exp(y_train), np.exp(pred)))))
print('train rmse: {}'.format(int(
    mean_squared_error(np.exp(y_train), np.exp(pred), squared=False))))
print('train r2: {}'.format(
    r2_score(np.exp(y_train), np.exp(pred))))
print()

# make predictions for test set
pred = price_pipe.predict(X_test)

# determine mse, rmse and r2
print('test mse: {}'.format(int(
    mean_squared_error(np.exp(y_test), np.exp(pred)))))
print('test rmse: {}'.format(int(
    mean_squared_error(np.exp(y_test), np.exp(pred), squared=False))))
print('test r2: {}'.format(
    r2_score(np.exp(y_test), np.exp(pred))))
print()

print('Average lodge price: ', int(np.exp(y_train).median()))

In [ ]:
# let's evaluate our predictions respect to the real sale price
plt.scatter(y_test, price_pipe.predict(X_test))
plt.xlabel('True Lodge Price')
plt.ylabel('Predicted Lodge Price')
plt.title('Evaluation of Lasso Predictions')

In [ ]:
# let's evaluate the distribution of the errors: 
# they should be fairly normally distributed

y_test.reset_index(drop=True, inplace=True)

preds = pd.Series(price_pipe.predict(X_test))

errors = y_test - preds
errors.hist(bins=30)
plt.show()

In [ ]:
# now let's save the scaler

joblib.dump(price_pipe, 'price_pipe.joblib') 